## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.65,115.33,63.00,64,0,3.67,clear sky
1,1,Hobart,AU,-42.88,147.33,70.00,45,75,12.75,broken clouds
2,2,Wajima,JP,37.40,136.90,44.60,65,75,6.93,broken clouds
3,3,Rikitea,PF,-23.12,-134.97,75.83,80,69,17.90,broken clouds
4,4,Punta Arenas,CL,-53.15,-70.92,39.20,71,40,31.09,scattered clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Rikitea,PF,-23.12,-134.97,75.83,80,69,17.90,broken clouds
11,11,Hilo,US,19.73,-155.09,80.60,57,40,9.17,scattered clouds
13,13,Touros,BR,-5.20,-35.46,78.80,74,40,9.17,scattered clouds
15,15,Vaini,TO,-21.20,-175.20,84.20,79,40,23.04,scattered clouds
18,18,Atuona,PF,-9.80,-139.03,78.80,74,60,21.18,light rain
...,...,...,...,...,...,...,...,...,...,...
682,682,Bac Lieu,VN,9.29,105.72,80.85,75,1,5.88,clear sky
684,684,Lakes Entrance,AU,-37.88,147.98,82.99,54,99,5.01,moderate rain
685,685,Boca Do Acre,BR,-8.75,-67.40,77.18,78,100,2.57,overcast clouds
686,686,Poya,NC,-21.35,165.15,80.85,52,70,11.43,broken clouds


In [15]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                158
City                   158
Country                158
Lat                    158
Lng                    158
Max Temp               158
Humidity               158
Cloudiness             158
Wind Speed             158
Current Description    158
dtype: int64

In [44]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.dropna()
clean_df.count()

City_ID                158
City                   158
Country                158
Lat                    158
Lng                    158
Max Temp               158
Humidity               158
Cloudiness             158
Wind Speed             158
Current Description    158
dtype: int64

In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Rikitea,PF,75.83,broken clouds,-23.12,-134.97,
11,Hilo,US,80.60,scattered clouds,19.73,-155.09,
13,Touros,BR,78.80,scattered clouds,-5.20,-35.46,
15,Vaini,TO,84.20,scattered clouds,-21.20,-175.20,
18,Atuona,PF,78.80,light rain,-9.80,-139.03,
19,Faanui,PF,81.25,broken clouds,-16.48,-151.75,
21,Oistins,BB,77.00,scattered clouds,13.07,-59.53,
23,Manacapuru,BR,75.20,thunderstorm,-3.30,-60.62,
28,Kavieng,PG,82.69,moderate rain,-2.57,150.80,
29,Souillac,MU,75.20,shower rain,-20.52,57.52,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [31]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Rikitea,PF,75.83,broken clouds,-23.12,-134.97,Pension Maro'i
11,Hilo,US,80.60,scattered clouds,19.73,-155.09,Hilo Hawaiian Hotel
13,Touros,BR,78.80,scattered clouds,-5.20,-35.46,INN NEW HORIZON
15,Vaini,TO,84.20,scattered clouds,-21.20,-175.20,Keleti Beach Resort
18,Atuona,PF,78.80,light rain,-9.80,-139.03,Villa Enata


In [32]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Rikitea,PF,75.83,broken clouds,-23.12,-134.97,Pension Maro'i
11,Hilo,US,80.60,scattered clouds,19.73,-155.09,Hilo Hawaiian Hotel
13,Touros,BR,78.80,scattered clouds,-5.20,-35.46,INN NEW HORIZON
15,Vaini,TO,84.20,scattered clouds,-21.20,-175.20,Keleti Beach Resort
18,Atuona,PF,78.80,light rain,-9.80,-139.03,Villa Enata
...,...,...,...,...,...,...,...
682,Bac Lieu,VN,80.85,clear sky,9.29,105.72,Khách sạn Sài Gòn Bạc Liêu
684,Lakes Entrance,AU,82.99,moderate rain,-37.88,147.98,The Esplanade Resort & Spa
685,Boca Do Acre,BR,77.18,overcast clouds,-8.75,-67.40,Hotel Linhares
686,Poya,NC,80.85,broken clouds,-21.35,165.15,


In [34]:
# 8a. Create the output File (CSV)
output_data_file = "vacation_search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [40]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [42]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))